In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import math
import numpy as np

!pip install transformers

from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers import BertTokenizer, BertModel


!pip install tqdm
!pip install datasets
import json
from tqdm import tqdm
from datasets import Dataset as ds
import random
import json

import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset

from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mounted at /content/drive
cuda


In [ ]:
!pip install datasets

In [ ]:
import json
from datasets import Dataset

# Load JSON file
dataset_te  = None
with open('/content/drive/MyDrive/IRE/Project/top_2000_telugu_samples.json', 'r', encoding='utf-8') as file:
    dataset_te = json.load(file)

# Convert JSON data to Hugging Face Dataset
# Assuming json_data is a list of dictionaries
# dataset_te = Dataset.from_dict(json_data)

# If json_data is a dictionary with lists as values (each list representing a column)
# you can use it directly in Dataset.from_dict


In [ ]:
type(dataset_te[0])
dataset_te[0]

{'document_plaintext': 'మెదక్ జిల్లా తెలంగాణ రాష్ట్రంలోని 31 జిల్లాలలో ఒకటి.[1]\n\nమెదక్\u200c పట్టణం హైదరాబాదుకు 100 కి మీ ల దూరంలో ఉంది. మెదక్ జిల్లాకు ఇది ముఖ్యపట్టణం. \nజనాభా లెక్కలు\n1981 జనాభా లెక్కల ప్రకారం ఈ జిల్లా జనాభా 18,07,139, స్త్రీ, పురుషుల నిష్పత్తి 98:100, అక్షరాస్యత 21.38 శాతం.(మూలం: ఆంధ్రప్రదేశ్ దర్శిని  1985)\n2011 జనగణన ప్రకారం జిల్లా జనాభా 30,31,877. జనసాంద్రత 313/కి.మీ² (811/చ.మై), పురుషులు 15,24,187, స్త్రీలు 15,07,690. 2001 భారతీయ జనాభా గణనను అనుసరించి అక్షరాస్యత 66 %. పురుషుల అక్షరాస్యత 74%. స్త్రీల అక్షరాస్యత 57%. జనాభాలో 6 సంవత్సరాలకు దిగువన ఉన్న వారి శాతం 13%.\nచరిత్ర\nపూర్వం సిద్దాపూర్</i>\u200c అని పిలువబడే నేటి మెదక్, కాకతీయుల కాలంలో ఉచ్ఛస్థితిలో ఉండేది. ఆ కాలం నాటి దుర్గం మెదక్ లో ఉంది. మెతుకుసీమగా తరువాతి కాలంలో పిలువబడేది. 1830లో కాశీయాత్రలో భాగంగా జిల్లాలోని పలు గ్రామాలు, పట్టణాలలో మజిలీ చేస్తూ ప్రయాణించిన యాత్రా చరిత్రకారుడు ఏనుగుల వీరాస్వామయ్య ఆనాడు ఈ ప్రాంతపు స్థితిగతుల గురించి వ్యాఖ్యానించారు. హైదరాబాద్ రాజ్యంలో, కృష్ణానది దాటినది మొదలుకొని హైదరా

In [ ]:
passage_list = []
question_list = []
for i in(dataset_te):
    passage_list.append(i['document_plaintext'])
    question_list.append(i['question_text'])

In [ ]:
print(len(passage_list))
passage_list[0]

2000


'మెదక్ జిల్లా తెలంగాణ రాష్ట్రంలోని 31 జిల్లాలలో ఒకటి.[1]\n\nమెదక్\u200c పట్టణం హైదరాబాదుకు 100 కి మీ ల దూరంలో ఉంది. మెదక్ జిల్లాకు ఇది ముఖ్యపట్టణం. \nజనాభా లెక్కలు\n1981 జనాభా లెక్కల ప్రకారం ఈ జిల్లా జనాభా 18,07,139, స్త్రీ, పురుషుల నిష్పత్తి 98:100, అక్షరాస్యత 21.38 శాతం.(మూలం: ఆంధ్రప్రదేశ్ దర్శిని  1985)\n2011 జనగణన ప్రకారం జిల్లా జనాభా 30,31,877. జనసాంద్రత 313/కి.మీ² (811/చ.మై), పురుషులు 15,24,187, స్త్రీలు 15,07,690. 2001 భారతీయ జనాభా గణనను అనుసరించి అక్షరాస్యత 66 %. పురుషుల అక్షరాస్యత 74%. స్త్రీల అక్షరాస్యత 57%. జనాభాలో 6 సంవత్సరాలకు దిగువన ఉన్న వారి శాతం 13%.\nచరిత్ర\nపూర్వం సిద్దాపూర్</i>\u200c అని పిలువబడే నేటి మెదక్, కాకతీయుల కాలంలో ఉచ్ఛస్థితిలో ఉండేది. ఆ కాలం నాటి దుర్గం మెదక్ లో ఉంది. మెతుకుసీమగా తరువాతి కాలంలో పిలువబడేది. 1830లో కాశీయాత్రలో భాగంగా జిల్లాలోని పలు గ్రామాలు, పట్టణాలలో మజిలీ చేస్తూ ప్రయాణించిన యాత్రా చరిత్రకారుడు ఏనుగుల వీరాస్వామయ్య ఆనాడు ఈ ప్రాంతపు స్థితిగతుల గురించి వ్యాఖ్యానించారు. హైదరాబాద్ రాజ్యంలో, కృష్ణానది దాటినది మొదలుకొని హైదరాబాద్ నగరం వరకూ ఉన్న ప్ర

In [ ]:
print(len(question_list))
question_list[0]

2000


'మెదక్ నగర విస్తీర్ణం ఎంత?'

In [ ]:
class Train_Dataset(Dataset):
    def _init_(self, src_Lines, trg_Lines):
        self.Passages = src_Lines
        self.Questions = trg_Lines

    def _len_(self):
        return len(self.TrainLines)

    def _getitem_(self, index):
        return self.Passages[index], self.Questions[index]

class Val_Dataset(Dataset):
    def _init_(self, src_Lines, trg_Lines):
        self.Passages = src_Lines
        self.Questions = trg_Lines

    def _len_(self):
        return len(self.TrainLines)

    def _getitem_(self, index):
        return self.Passages[index], self.Questions[index]

In [ ]:
Train_dataset = Train_Dataset(passage_list[:1500], question_list[:1500])

Train_dataloader = DataLoader(Train_dataset, batch_size = 8)

In [ ]:
for x, y in Train_dataloader:
    print(y)
    break

JSON file loaded successfully into a dictionary.
JSON file loaded successfully into a dictionary.
len(train_dict):  596
len(test_dict):  149


In [3]:
class Encoder(nn.Module):
    def __init__(self, model_name):
        super(Encoder, self).__init__()
        # cfg_name = "bert-base-uncased"
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        cfg = AutoConfig.from_pretrained(model_name)
        # self.BertModel= AutoModel.from_pretrained(model_name, config=cfg) # , project_dim=projection_dim, **kwargs
        self.BertModel = BertModel.from_pretrained(model_name)

    def forward(self, x):
        # inputs = tokenizer(x, return_tensors='pt')
        # print("x = ", x)
        # input_ids = self.tokenizer.encode(x, add_special_tokens=True, return_tensors='pt')
        # print("tokenized x shape = ", input_ids.shape)
        # input_ids = x.unsqueeze(0)
        # print("tokenized x = ", input_ids)
        tokenized_batch = self.tokenizer(x, padding=True, truncation=True, add_special_tokens=True, return_tensors='pt')

        input_ids = tokenized_batch['input_ids']
        # print("input_ids shape  ", input_ids.shape)
        attention_mask = tokenized_batch['attention_mask']
        token_type_ids = tokenized_batch['token_type_ids']
        # print("outputs shape = ", outputs.shape)

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = self.BertModel(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        # print("cls shape = ", cls_embeddings.shape)
        # cls_embeddings = cls_embeddings.reshape(cls_embeddings.size()[1])
        # print("output shape= ", outputs.shape)
        # print("outputs = ", outputs)
        # out  = outputs[0]
        # print("first output shape= ", cls_embeddings.shape)
        # print("first output = ", cls_embeddings)
        return cls_embeddings

In [4]:
class BiEncoder(nn.Module):
    def __init__(self, model_name):
        super(BiEncoder, self).__init__()
        self.model_name = model_name
        self.Q_Encoder = Encoder(model_name)
        self.P_Encoder = Encoder(model_name)

    def forward(self, question, PositiveDoc):
        q_emb = self.Q_Encoder(question)
        p_emb = self.P_Encoder(PositiveDoc)

        # neg_embs = []
        # for NegativeDoc in NegativeDocs:
        #     n_emb = self.P_Encoder(NegativeDoc)
        #     neg_embs.append(n_emb)

        return  q_emb, p_emb

In [ ]:
EPOCHS = 5
model_name = "bert-base-multilingual-uncased"
model = BiEncoder(model_name).to(device)
# model

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [5]:
def Train_BiEncoder(p_model, Dataloader, loss_func, optimizer):
    TotalLoss = 0
    for q, p_ctx in tqdm(Dataloader):

        optimizer.zero_grad()
        q = list(q)
        p_ctx = list(p_ctx)
        # q = q.to(device)
        # p_ctx = p_ctx.to(device)
        q_emb, p_emb = p_model(q, p_ctx)

        # print("q: ", q_emb.shape)
        # print(p_emb.shape)
        p_emb = torch.transpose(p_emb, 0, 1)
        r = torch.matmul(q_emb, p_emb)
        softmax_scores = F.log_softmax(r, dim=1)
        positive_idx_per_question = [x for x in range(softmax_scores.size(0))]

        # list(map(lambda x : x in for x in range(softmax_scores.size(0))))

        loss = F.nll_loss(
            softmax_scores,
            torch.tensor(positive_idx_per_question).to(softmax_scores.device),
            reduction="mean",
        )
        # loss = loss_func(q_emb, p_emb, n_embs)
        TotalLoss  = TotalLoss + loss.item()

        loss.backward()
        optimizer.step()

    return TotalLoss

In [6]:
import matplotlib.pyplot as plt

def LoadOrTrainModel(p_model, Dataloader, loss_function, optimizer, SavedModel = None):
    # Loading Model

    if SavedModel != None:
        p_model = torch.load(SavedModel).to(device)
        print("Model Loaded succesfully")
        return p_model

    # Training Model
    minTrainLoss = float('inf')
    loss = []
    for epoch in range(EPOCHS):
        TrainLoss = Train_BiEncoder(p_model, Dataloader, loss_function, optimizer)

        print("EPOCH - ", epoch+1)
        print("TRAIN LOSS=", TrainLoss)
        print("")
        if TrainLoss < minTrainLoss:
            torch.save(p_model, 'DPR.pt')
            minTrainLoss = TrainLoss
            print("Model Saved")

        loss.append(TrainLoss)

    epochs = [i+1 for i in range(EPOCHS)]
    plt.plot(epochs, loss)
    plt.xlabel("EPOCHS")
    plt.ylabel("LOSS")

    plt.show()

    p_model = torch.load('DPR.pt').to(device)
    return p_model


In [ ]:
# loss_func = CustomLoss()
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

model = LoadOrTrainModel(model, train_dataloader, loss_func, optimizer, "/content/drive/MyDrive/IRE/Project/DPR.pt") #, "/content/drive/MyDrive/IRE/DPR.pt"
model = model.to(device)

In [ ]:
train_dataloader_new, train_dataset_new = get_loader(train_dataset_object, train_split_type, 1)

In [43]:
def ComputePassageEmbeddings(p_model, Dataloader, Dataloader2 = None):
    Pass_Emb = []
    Orginal_Passages = []
    Orginal_Questions = []
    for q, p_ctx in tqdm(Dataloader):
        q = q[0]
        p_ctx = p_ctx[0]
        # print(type(q))
        # print(type(p_ctx))
        # print(p_ctx)
        # print(q)
        Orginal_Passages.append(p_ctx)
        Orginal_Questions.append(q)
        out = p_model.P_Encoder(p_ctx)
        out = out.cpu().detach()
        Pass_Emb.extend(out)
        # break

    if Dataloader2 is not None:
        for q, p_ctx in tqdm(Dataloader2):
            q = q[0]
            p_ctx = p_ctx[0]
            # print(type(q))
            # print(type(p_ctx))
            # print(p_ctx)
            # print(q)
            Orginal_Passages.append(p_ctx)
            Orginal_Questions.append(q)
            out = p_model.P_Encoder(p_ctx)
            out = out.cpu().detach()
            Pass_Emb.extend(out)

    return Orginal_Passages, Orginal_Questions, Pass_Emb


In [ ]:
Org_Docs, SavedPassageEmbeddings = ComputePassageEmbeddings(model, train_dataloader_new)

**FAISS Testing**

In [ ]:
import torch
from datasets import Dataset

# Convert each tensor to a numpy array
numpy_list = [tensor.numpy().astype(np.float32) for tensor in SavedPassageEmbeddings]

# Creating the dictionary for the dataset
data_dict = {
    "embeddings": numpy_list,
    "Org_Docs": Org_Docs,
}

# Creating a Hugging Face dataset
embeddings_dataset = Dataset.from_dict(data_dict)

In [ ]:
np.array(numpy_list).shape

(596, 768)

In [ ]:
np.array(numpy_list)[:2]

array([[-0.12955084,  0.13028407, -0.07223221, ..., -0.07677489,
         0.25569654, -0.0718286 ],
       [-0.09987344,  0.0948275 , -0.18100394, ..., -0.12100865,
         0.20280051,  0.01415021]], dtype=float32)

In [ ]:
embeddings_dataset

Dataset({
    features: ['embeddings', 'Org_Docs'],
    num_rows: 596
})

In [ ]:
type(embeddings_dataset['embeddings'][0])

list

In [ ]:
len(embeddings_dataset['embeddings'][0])

768

In [ ]:
embeddings_dataset['embeddings'][0]

In [ ]:
!pip install faiss-gpu


In [ ]:
type(embeddings_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
embeddings_dataset

Dataset({
    features: ['embeddings', 'Org_Docs'],
    num_rows: 596
})

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['embeddings', 'Org_Docs'],
    num_rows: 596
})

In [ ]:
np.array(embeddings_dataset['embeddings'][0]).shape

(768,)

In [ ]:
question_1 = "विद्याधर शास्त्री को भारत के राष्ट्रपति द्वारा किस उपाधि से सम्मानित किया गया था?"
question_2 = "चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?"
question_3 = "अर्नेस्ट रदरफोर्ड, किस पेशे से सम्बंधित थे?"
question_4 = "चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?"
# question_embedding = model.Q_Encoder(question_2).cpu().detach().numpy().astype(np.float32).transpose()
question_embedding = model.Q_Encoder(question_2).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [ ]:
np.array(numpy_list).shape

(596, 768)

In [ ]:
import faiss
import numpy as np

# Assuming `dataset` is your dataset with document embeddings
document_embeddings = np.array(numpy_list)

# Normalize the document embeddings
faiss.normalize_L2(document_embeddings)

# Creating a FAISS index for cosine similarity
d = document_embeddings.shape[1]  # Dimension of the embeddings
index = faiss.index_factory(d, "Flat", faiss.METRIC_INNER_PRODUCT)
index.add(document_embeddings)

# Assuming `encoded_questions` is a numpy array of your question embeddings
encoded_questions = [question_embedding]

# Number of top documents to retrieve
k = 5

for question in encoded_questions:
    # Normalize the question embedding
    normalized_question = question / np.linalg.norm(question)
    normalized_question = normalized_question.reshape(1, -1).astype(np.float32)

    # Search the index
    distances, indices = index.search(normalized_question, k)

    # Process the results for this question
    print(f"Top {k} documents for a given question:")
    for i, idx in enumerate(indices[0]):
        print(f"Document {idx} with similarity: {distances[0][i]}")

    # result = 1 - spatial.distance.cosine(document_embeddings[0], encoded_questions)
    # print('Distance by normal cosine similarity:{}'.format(result))


Top 5 documents for a given question:
Document 516 with similarity: -0.02223547361791134
Document 148 with similarity: -0.02980859763920307
Document 549 with similarity: -0.035037778317928314
Document 235 with similarity: -0.03987196087837219
Document 502 with similarity: -0.040230005979537964


In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    # print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    # print(f"TITLE: {row.title}")
    print(f"Org_Doc: {row.Org_Docs}")
    print("=" * 50)
    # print(row)
    print()


SCORE: 505.6034240722656
Org_Doc: उर्दू भाषा हिन्द आर्य भाषा है। उर्दू भाषा हिन्दुस्तानी भाषा की एक मानकीकृत रूप मानी जाती है। उर्दू में संस्कृत के तत्सम शब्द न्यून हैं और अरबी-फ़ारसी और संस्कृत से तद्भव शब्द अधिक हैं। ये मुख्यतः दक्षिण एशिया में बोली जाती है। यह भारत की शासकीय भाषाओं में से एक है तथा पाकिस्तान की राष्ट्रभाषा है। [1] इस के अतिरिक्त भारत के राज्य जम्मू और कश्मीर की मुख्य प्रशासनिक भाषा है। साथ ही तेलंगाना, दिल्ली, बिहार[2] और उत्तर प्रदेश की अतिरिक्त शासकीय [3]भाषा है।
'उर्दू' शब्द की व्युत्पत्ति
'उर्दू' शब्द मूलतः तुर्की भाषा का है तथा इसका अर्थ है- 'शाही शिविर’ या ‘खेमा’(तम्बू)। तुर्कों के साथ यह शब्द भारत में आया और इसका यहाँ प्रारम्भिक अर्थ खेमा या सैन्य पड़ाव था। शाहजहाँ ने दिल्ली में लालकिला बनवाया। यह भी एक प्रकार से ‘उर्दू’ (शाही और सैन्य पड़ाव) था, किन्तु बहुत बड़ा था। अतः इसे ‘उर्दू’ न कहकर ‘उर्दू ए मुअल्ला’ कहा गया तथा यहाँ बोली जाने वाली भाषा- ‘ज़बान ए उर्दू ए मुअल्ला’ (श्रेष्ठ शाही पड़ाव की भाषा) कहलाई। भाषा विशेष के अर्थ में ‘उर्दू’ शब्द इस ‘ज़बान ए उर्दू 

In [ ]:
type(Org_Docs[0])
Org_Docs[0]

'भारत के पश्चिमी तट पर स्थित मुंंबई (पूर्व नाम बम्बई), भारतीय राज्य महाराष्ट्र की राजधानी है। इसकी अनुमानित जनसंख्या ३ करोड़ २९ लाख है जो देश की पहली सर्वाधिक आबादी वाली नगरी है।[1] इसका गठन लावा निर्मित सात छोटे-छोटे द्वीपों द्वारा हुआ है एवं यह पुल द्वारा प्रमुख भू-खंड के साथ जुड़ा हुआ है। मुंबई बन्दरगाह भारतवर्ष का सर्वश्रेष्ठ सामुद्रिक बन्दरगाह है। मुम्बई का तट कटा-फटा है जिसके कारण इसका पोताश्रय प्राकृतिक एवं सुरक्षित है। यूरोप, अमेरिका, अफ़्रीका आदि पश्चिमी देशों से जलमार्ग या वायुमार्ग से आनेवाले जहाज यात्री एवं पर्यटक सर्वप्रथम मुम्बई ही आते हैं, इसलिए मुम्बई को भारत का प्रवेशद्वार कहा जाता है।\nमुम्बई भारत का सर्ववृहत्तम वाणिज्यिक केन्द्र है। जिसकी भारत के सकल घरेलू उत्पाद में 5% की भागीदारी है।[2] यह सम्पूर्ण भारत के औद्योगिक उत्पाद का 25%, नौवहन व्यापार का 40%, एवं भारतीय अर्थ व्यवस्था के पूंजी लेनदेन का 70% भागीदार है।[3] मुंबई विश्व के सर्वोच्च दस वाणिज्यिक केन्द्रों में से एक है।[4] भारत के अधिकांश बैंक एवं सौदागरी कार्यालयों के प्रमुख कार्यालय एवं कई महत्वपूर्ण आर्थिक सं

In [46]:
def RetrieveTopKDocuments(p_model, Question, SavedPassageEmbeddings, Org_Docs, TopK):
    q_emb = p_model.Q_Encoder(Question)
    q_emb = q_emb.cpu().detach()
    sim_Dict = {}

    retrievedDocs = []
    retrievedDocId = []
    for i in range(len(SavedPassageEmbeddings)):
        sim_Dict[i] = CalculateSim(q_emb, SavedPassageEmbeddings[i])

    sorted_SimDict = dict(sorted(sim_Dict.items(), key=lambda x:x[1], reverse=True)[:TopK])

    for i, k in enumerate(sorted_SimDict.keys()):
        print("RANK -", i+1, " -> Document - ", k)
        # print(Org_Docs[k])
        # print()
        retrievedDocs.append(Org_Docs[k])
        retrievedDocId.append(k)


    return retrievedDocs, retrievedDocId

def CalculateSim(a, b):
        return torch.matmul(a, b.reshape(b.shape[0], -1))
        # return torch.matmul(a, b.transpose(0, 1))
        # a = a.numpy()
        # b = b.numpy()
        # return np.dot(a, b)

In [ ]:
enc = Encoder("bert-base-multilingual-uncased").to(device)

li = ["यह हिंदी में एक उदाहरण पाठ है जिसके लिए हम एम्बेडिंग चाहते हैं।", "यह हिंदी में एक उदाहरण पाठ है जिसके लिए हम एम्बेडिंग चाहते हैं।", "यह हिंदी में एक उदाहरण पाठ है जिसके लिए हम एम्बेडिंग चाहते हैं।"]

t = enc("यह हिंदी में एक उदाहरण पाठ है जिसके लिए हम एम्बेडिंग चाहते हैं।")
print(t.shape, t)

In [ ]:
print(enc)

In [ ]:
print(train_dataset_object["question"][0])
print(train_dataset_object["positivie_context"][0])
print()
print()
# print(train_dataset_object["negative_context"][0])

चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?
चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहासिक उललेखों के लिये प्रख्यात है। चीन का प्राचीन साहित्य "पाँच क्लासिकल" के रूप में उपलब्ध होता है जिसके प्राचीनतम भाग का ईसा के पूर्व लगभग 15वीं शताब्दी माना जाता है। इसमें इतिहास (शू चिंग), प्रशस्तिगीत (शिह छिंग), परिवर्तन (ई चिंग), विधि विधान (लि चि) तथा कनफ्यूशियस (552-479 ई.पू.) द्वारा संग्रहित वसंत और शरद-विवरण (छुन छिउ) नामक तत्कालीन इतिहास शामिल हैं जो छिन राजवंशों के पूर्व का एकमात्र ऐतिहासिक संग्रह है। पूर्वकाल में शासनव्यवस्था चलाने के लिये राज्य के पदाधिकारियों को कनफ्यूशिअस धर्म में पारंगत होना आवश्यक था, इससे सरकारी परीक्षाओं के लिये इन ग्रंथों का अध्ययन अनिवार्य कर दिया गया था।
कनफ्यूशिअस के अतिरिक्त चीन में लाओत्स, चुआंगत्स और मेन्शियस आदि अनेक दार्शनिक हो गए हैं जिनके साहित्य ने चीनी जनजीवन को प्रभावित किया है।
 जनकवि चू य्वान 
चू य्वान् (340-278 ई.पू.) चीन के सर्वप्रथम जनकवि माने जाते हैं। वे चू राज्य के निवासी देशभक्त मंत्री थे। राज्यकर्मचारियों के षड्यंत्र के कारण दुश्चरित्रता का 

In [ ]:
print(test_dataset_object.keys())

dict_keys(['question', 'positivie_context', 'negative_context'])


In [ ]:
Question = "चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?"

dos = RetrieveTopKDocuments(model, Question, SavedPassageEmbeddings, Org_Docs, 5)

RANK - 1  -> Document -  293
चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहासिक उललेखों के लिये प्रख्यात है। चीन का प्राचीन साहित्य "पाँच क्लासिकल" के रूप में उपलब्ध होता है जिसके प्राचीनतम भाग का ईसा के पूर्व लगभग 15वीं शताब्दी माना जाता है। इसमें इतिहास (शू चिंग), प्रशस्तिगीत (शिह छिंग), परिवर्तन (ई चिंग), विधि विधान (लि चि) तथा कनफ्यूशियस (552-479 ई.पू.) द्वारा संग्रहित वसंत और शरद-विवरण (छुन छिउ) नामक तत्कालीन इतिहास शामिल हैं जो छिन राजवंशों के पूर्व का एकमात्र ऐतिहासिक संग्रह है। पूर्वकाल में शासनव्यवस्था चलाने के लिये राज्य के पदाधिकारियों को कनफ्यूशिअस धर्म में पारंगत होना आवश्यक था, इससे सरकारी परीक्षाओं के लिये इन ग्रंथों का अध्ययन अनिवार्य कर दिया गया था।
कनफ्यूशिअस के अतिरिक्त चीन में लाओत्स, चुआंगत्स और मेन्शियस आदि अनेक दार्शनिक हो गए हैं जिनके साहित्य ने चीनी जनजीवन को प्रभावित किया है।
 जनकवि चू य्वान 
चू य्वान् (340-278 ई.पू.) चीन के सर्वप्रथम जनकवि माने जाते हैं। वे चू राज्य के निवासी देशभक्त मंत्री थे। राज्यकर्मचारियों के षड्यंत्र के कारण दुश्चरित्रता का दोषारोपण कर उन

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np

In [ ]:
import faiss
dataSetI = [.1, .2, .3]
dataSetII = [.4, .5, .6]
#dataSetII = [.1, .2, .3]

x = np.array([dataSetI]).astype(np.float32)
q = np.array([dataSetII]).astype(np.float32)
index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(x)
index.add(x)
faiss.normalize_L2(q)
distance, index = index.search(q, 5)
print('Distance by FAISS:{}'.format(distance))

#To Tally the results check the cosine similarity of the following example

from scipy import spatial

result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
print('Distance by cosine:{}'.format(result))

Distance by FAISS:[[ 9.7463191e-01 -3.4028235e+38 -3.4028235e+38 -3.4028235e+38
  -3.4028235e+38]]
Distance by FAISS:0.9746318461970764


In [39]:
def GetTestPerformance(p_model, Org_Questions, SavedPassageEmbeddings, OrgDocs, TopK):

    Num_Correct = 0;
    Reciprocal_Rank = 0
    idx = 0
    for i in tqdm(range(len(Org_Questions))):
        q = Org_Questions[i]
        # p_ctx = p_ctx[0]
        y_true = i

        _, retrieved_DocId = RetrieveTopKDocuments(p_model, q, SavedPassageEmbeddings, OrgDocs, TopK)

        for i in range(TopK):
            if retrieved_DocId[i] == y_true:
                Num_Correct += 1
                Reciprocal_Rank += 1/(i+1)
                break

        idx += 1
        # Orginal_Passages.append(p_ctx)
        # out = p_model.P_Encoder(p_ctx)
        # out = out.cpu().detach()
        # Pass_Emb.append(out)
        # break

    Accuracy = Num_Correct/(idx+1)
    MRR = Reciprocal_Rank/(idx+1)

    return Accuracy, MRR

**CHAII Dataset Inferencing**

In [8]:
# file_path = r"/content/drive/MyDrive/IRE/project/train_chaii_hi_1.json"
# train_file_path = r"/content/drive/MyDrive/IRE/project/train.json"
# test_file_path = r"/content/drive/MyDrive/IRE/project/test.json"

file_path = r"/content/drive/MyDrive/IRE/Project/train_chaii_hi_1.json"
train_file_path = r"/content/drive/MyDrive/IRE/Project/train.json"
test_file_path = r"/content/drive/MyDrive/IRE/Project/test.json"

In [9]:
def load_data(file):

  # Specify the path to your JSON file
  json_file_path = file
  # Initialize an empty dictionary
  data = {}

  # Load the JSON data into the dictionary
  try:
      with open(json_file_path, "r") as json_file:
          data = json.load(json_file)
      print("JSON file loaded successfully into a dictionary.")
  except FileNotFoundError:
      print(f"Error: JSON file '{json_file_path}' not found.")
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")

  return data

train_dict = load_data(train_file_path)
test_dict = load_data(test_file_path)
print("len(train_dict): ", len(train_dict))
print("len(test_dict): ", len(test_dict))

chai_dataset = {
  "train": {"question": [], "positivie_context": [], "negative_context": []},
  "test": {"question": [], "positivie_context": [], "negative_context": []},
}

def get_data(split_type, dict_type, chai_dataset, k):

  question_list = []
  positive_contest_list = []
  negative_contest_list = []
  random_seed = 42
  random.seed(random_seed)
  for i in dict_type:
    question_list.append(i['question'])
    positive_contest_list.append(i["positive_ctxs"][0]['text'])

  for i in range(len(positive_contest_list)):
    list1 =[]
    for j in range(len(positive_contest_list)):
      if(i!=j):
        list1.append(positive_contest_list[j])
    list1 = random.choices(list1, k=k)
    negative_contest_list.append(list1[:])

  for i in range(len(question_list)):
    chai_dataset[split_type]["question"] = question_list
    chai_dataset[split_type]["positivie_context"] = positive_contest_list
    chai_dataset[split_type]["negative_context"] = negative_contest_list

#build train
get_data("train", train_dict, chai_dataset, 10)

#build test
get_data("test", test_dict, chai_dataset, 10)

class chaiDataset(Dataset):
  def __init__(self, dataset_object, split_type):
    #convert list to pandas datafraame
    #create new df
    # split_type = 'train'

    self.df = pd.DataFrame({'question':dataset_object[split_type]['question'],'positivie_context':dataset_object[split_type]['positivie_context'],'negative_context':dataset_object[split_type]['negative_context']})
    # print ("df:",self.df)

    # Get question columns
    self.question = self.df["question"]
    # Get positive_context columns
    self.positivie_context = self.df["positivie_context"]
    # Get the negative_context columns
    self.negative_context = self.df["negative_context"]

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    question = self.question[index]
    positive_context = self.positivie_context[index]
    negative_context = self.negative_context[index]

    return question, positive_context

def get_loader(
    dataset_object,
    split_type,
    batch_size=1,
    shuffle=True,
):
  dataset = chaiDataset(chai_dataset, split_type)

  loader = DataLoader(
      dataset=dataset,
      batch_size=batch_size,
      shuffle=shuffle,
  )

  return loader, dataset

JSON file loaded successfully into a dictionary.
JSON file loaded successfully into a dictionary.
len(train_dict):  596
len(test_dict):  149


In [10]:
test_dataset_object = chai_dataset["test"]
train_dataset_object = chai_dataset["train"]
test_split_type = "test"
train_split_type = "train"
batch_size = 8
test_dataloader, test_dataset = get_loader(test_dataset_object, test_split_type, batch_size)
train_dataloader, train_dataset = get_loader(train_dataset_object, train_split_type, batch_size)

In [11]:
EPOCHS = 5
model_name = "bert-base-multilingual-uncased"
model = BiEncoder(model_name).to(device)
# model

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [12]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

model = LoadOrTrainModel(model, train_dataloader, loss_func, optimizer, "/content/drive/MyDrive/IRE/Project/DPR.pt") #, "/content/drive/MyDrive/IRE/Project/DPR.pt"
model = model.to(device)

Model Loaded succesfully


In [28]:
train_dataloader_new, train_dataset_new = get_loader(train_dataset_object, train_split_type, 1)

In [44]:
Org_Docs_Train, Org_Questions_Train, SavedPassageEmbeddings_Train = ComputePassageEmbeddings(model, train_dataloader_new)

100%|██████████| 596/596 [00:51<00:00, 11.66it/s]


In [45]:
Accuracy, MRR = GetTestPerformance(model, Org_Questions_Train, SavedPassageEmbeddings_Train, Org_Docs_Train, 5)
print()
print("Train Accuracy of Retrieved Documents :", Accuracy)
print("Train MRR of Retrieved Documents :", MRR)

100%|██████████| 596/596 [00:25<00:00, 22.99it/s]


Train Accuracy of Retrieved Documents : 0.8157453936348409
Train MRR of Retrieved Documents : 0.5783919597989943


In [35]:
test_dataloader_new, test_dataset_new = get_loader(test_dataset_object, test_split_type, 1)

In [58]:
Org_Docs_TT, Org_Questions_TT, SavedPassageEmbeddings_TT = ComputePassageEmbeddings(model, train_dataloader_new, test_dataloader_new)

100%|██████████| 149/149 [00:12<00:00, 12.19it/s]


In [41]:
Accuracy, MRR = GetTestPerformance(model, Org_Questions_TT, SavedPassageEmbeddings_TT, Org_Docs_TT, 5)
print()
print("Train + Test Accuracy of Retrieved Documents :", Accuracy)
print("Train + Test MRR of Retrieved Documents :", MRR)

100%|██████████| 745/745 [00:45<00:00, 16.27it/s]


Train + Test Accuracy of Retrieved Documents : 0.7466487935656837
Train + Test MRR of Retrieved Documents : 0.5149016979445923


In [52]:
question = "चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?"
print("Question -", question)
Docs, Doc_ID = RetrieveTopKDocuments(model, question, SavedPassageEmbeddings_Train, Org_Docs_Train, 5)
print()
print("First Retrieved Document - ")
print(Docs[0])


Question - चीन के सर्वप्रथम जनकवि किसे माना जाता हैं?
RANK - 1  -> Document -  406
RANK - 2  -> Document -  327
RANK - 3  -> Document -  434
RANK - 4  -> Document -  370
RANK - 5  -> Document -  316

First Retrieved Document - 
चीनी साहित्य अपनी प्राचीनता, विविधता और ऐतिहासिक उललेखों के लिये प्रख्यात है। चीन का प्राचीन साहित्य "पाँच क्लासिकल" के रूप में उपलब्ध होता है जिसके प्राचीनतम भाग का ईसा के पूर्व लगभग 15वीं शताब्दी माना जाता है। इसमें इतिहास (शू चिंग), प्रशस्तिगीत (शिह छिंग), परिवर्तन (ई चिंग), विधि विधान (लि चि) तथा कनफ्यूशियस (552-479 ई.पू.) द्वारा संग्रहित वसंत और शरद-विवरण (छुन छिउ) नामक तत्कालीन इतिहास शामिल हैं जो छिन राजवंशों के पूर्व का एकमात्र ऐतिहासिक संग्रह है। पूर्वकाल में शासनव्यवस्था चलाने के लिये राज्य के पदाधिकारियों को कनफ्यूशिअस धर्म में पारंगत होना आवश्यक था, इससे सरकारी परीक्षाओं के लिये इन ग्रंथों का अध्ययन अनिवार्य कर दिया गया था।
कनफ्यूशिअस के अतिरिक्त चीन में लाओत्स, चुआंगत्स और मेन्शियस आदि अनेक दार्शनिक हो गए हैं जिनके साहित्य ने चीनी जनजीवन को प्रभावित

In [57]:
question = "इंडोनेशिया की राजधानी क्या है"
print("Question -", question)
Docs, Doc_ID = RetrieveTopKDocuments(model, question, SavedPassageEmbeddings_Train, Org_Docs_Train, 5)
print()
print("First Retrieved Document - ")
print(Docs[0])

Question - इंडोनेशिया की राजधानी क्या है
RANK - 1  -> Document -  464
RANK - 2  -> Document -  30
RANK - 3  -> Document -  147
RANK - 4  -> Document -  576
RANK - 5  -> Document -  260

First Retrieved Document - 
इंडोनेशिया गणराज्य (दीपान्तर गणराज्य) दक्षिण पूर्व एशिया और ओशिनिया में स्थित एक देश है। १७५०८ द्वीपों वाले इस देश की जनसंख्या लगभग 26 करोड़ है, यह दुनिया का तीसरा  सबसे अधिक आबादी और दुनिया में सबसे बड़ी बौद्ध आबादी वाला देश है। देश की राजधानी जकार्ता है। देश की जमीनी सीमा पापुआ न्यू गिनी, पूर्वी तिमोर और मलेशिया के साथ मिलती है, जबकि अन्य पड़ोसी देशों सिंगापुर, फिलीपींस, ऑस्ट्रेलिया और भारत का अंडमान और निकोबार द्वीप समूह क्षेत्र शामिल है।
 इतिहास 
इसा पूर्व ४थी शताब्दी से ही इंडोनेशिया द्वीपसमूह एक महत्वपूर्ण व्यापारिक क्षेत्र रहा है। बुनी अथवा मुनि सभ्यता इंडोनेशिया की सबसे पुरानी सभ्यता है। ४थी शताब्दी इस्सा पूर्व तक ये सभ्यता काफी उन्नति कर चुकी थी। ये हिंदू धर्म मानते थे और ऋषि परम्परा का अनुकरण करते थे। अगले दो हजार साल तक इंडोनेशिया एक हिन्दू और बौद्ध देशों का समूह र

In [61]:
question = "चंद्रशेखर वेंकट रमन को भारत रत्न पुरस्कार कब मिला?"
print("Question -", question)
Docs, Doc_ID = RetrieveTopKDocuments(model, question, SavedPassageEmbeddings_TT, Org_Docs_TT, 5)
print()
print("First Retrieved Document - ")
print(Docs[0])

Question - चंद्रशेखर वेंकट रमन को भारत रत्न पुरस्कार कब मिला?
RANK - 1  -> Document -  81
RANK - 2  -> Document -  332
RANK - 3  -> Document -  499
RANK - 4  -> Document -  636
RANK - 5  -> Document -  552

First Retrieved Document - 
सीवी रमन (तमिल: சந்திரசேகர வெங்கட ராமன்) (७ नवंबर, १८८८ - २१ नवंबर, १९७०) भारतीय भौतिक-शास्त्री थे। प्रकाश के प्रकीर्णन पर उत्कृष्ट कार्य के लिये वर्ष १९३० में उन्हें भौतिकी का प्रतिष्ठित नोबेल पुरस्कार दिया गया। उनका आविष्कार उनके ही नाम पर रामन प्रभाव के नाम से जाना जाता है।[1] १९५४ ई. में उन्हें भारत सरकार द्वारा भारत रत्न की उपाधि से विभूषित किया गया तथा १९५७ में लेनिन शान्ति पुरस्कार प्रदान किया था।
परिचय चन्द्रशेखर वेंकटरमन का जन्म ७ नवम्बर सन् १८८८ ई. में तमिलनाडु के तिरुचिरापल्‍ली नामक स्थान में हुआ था। आपके पिता चन्द्रशेखर अय्यर एस. पी. जी. कॉलेज में भौतिकी के प्राध्यापक थे। आपकी माता पार्वती अम्मल एक सुसंस्कृत परिवार की महिला थीं। सन् १८९२ ई. मे आपके पिता चन्द्रशेखर अय्यर विशाखापतनम के श्रीमती ए. वी.एन. कॉलेज में भौतिकी और गणित के प्राध्यापक होक